## Setup

In [20]:
import os
import polars as pl

## paths

In [21]:
region_name = "Mitte"
mngmt_name = "Darmstadt"

In [22]:
pth_data = f"C:/Users/z187070/Documents/Projects/PAXCOUNTER/data/"

In [23]:
pth_master = pth_data + f"master/master/{region_name}/master_data/master_data.csv"
pth_ris = pth_data + f"ris/ris/{region_name}/{mngmt_name}/ris_data/ris_data.csv"
pth_pax = pth_data + f"pax/pax/{region_name}/{mngmt_name}/"

## Load data

In [24]:
df_master = pl.read_csv(pth_master, has_header = True, separator=";")

In [25]:
df_ris = pl.read_csv(pth_ris, has_header = True, separator=";")

In [26]:
df_pax = pl.read_csv(pth_pax + f"*.csv", has_header=True, separator=";")

## Descriptives

In [27]:
df_pax.head()

pax_counter_id,time_iot,data_pax
str,str,i64
"""244cab022ddf""","""2024-12-01 00:…",9
"""244cab022ddf""","""2024-12-01 00:…",5
"""244cab022ddf""","""2024-12-01 00:…",6
"""244cab022ddf""","""2024-12-01 00:…",6
"""244cab022ddf""","""2024-12-01 00:…",8


In [28]:
df_pax.select(pl.count())

count
u32
16058238


In [29]:
df_master.filter(pl.col("management_name") == region).head()

pax_counter_id,equipmentname,station_id,tplnr,tpname,station_name,station_ril100,management_name,region_name,federal_state_name,station_longitude,station_latitude,produktlinie,untersegment
str,str,i64,str,str,str,str,str,str,str,f64,f64,str,str
"""d48afc8ec697""","""DSA+_Typ2S_Gl2…",238,"""00238-01-B01""","""Bahnsteig 1, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""a0a3b38ca087""","""DSA+_Typ2S_Gl1…",238,"""00238-01-B01""","""Bahnsteig 1, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""244cab062d53""","""DSA+_Typ2S_Gl4…",238,"""00238-01-B02""","""Bahnsteig 2, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""3494545a211f""","""DSA+_Typ2S_Gl3…",238,"""00238-01-B02""","""Bahnsteig 2, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""a0a3b38c6c4f""","""DSA+_Typ2P_Gl4…",614,"""00614-01-B03""","""Bahnsteig 3, G…","""Biblis""","""FBL""","""Darmstadt""","""Mitte""","""Hessen""",8.450413,49.688881,"""Zubringerbahnh…","""Regionaler Zub…"


In [30]:
df_master.filter(pl.col("management_name") == region).select(pl.count())

count
u32
71


In [31]:
master_pcid = df_master.filter(pl.col("management_name") == mngmt_name).select(pl.col("pax_counter_id").unique()).to_series().to_list()

In [32]:
pax_pcid = df_pax.select(pl.col("pax_counter_id").unique()).to_series().to_list()

In [33]:
list(set(master_pcid) - set(pax_pcid))

[]

In [34]:
list(set(pax_pcid) - set(master_pcid))

[]

In [35]:
df_ris.head()

fahrtid,ereignis_station_id,zeit_echt,zeit_echt_verspaetung,gleis_echt,tpname_bahnsteig_echt,gattung,case
str,i64,str,i64,str,str,str,str
"""20241130-f7563…",238,"""2024-12-01 00:…",4,"""00238-01-B02-G…","""Bahnsteig 2, G…","""VIA""","""ankunft"""
"""20241130-ae4fd…",4614,"""2024-12-01 00:…",0,"""04614-01-B01-G…","""Bahnsteig 1, G…","""VIA""","""abfahrt"""
"""20241130-dc1e0…",238,"""2024-12-01 00:…",3,"""00238-01-B01-G…","""Bahnsteig 1, G…","""HLB""","""ankunft"""
"""20241130-67b5c…",238,"""2024-12-01 00:…",1,"""00238-01-B01-G…","""Bahnsteig 1, G…","""HLB""","""abfahrt"""
"""20241130-dc1e0…",238,"""2024-12-01 00:…",1,"""00238-01-B01-G…","""Bahnsteig 1, G…","""HLB""","""abfahrt"""


## joins

In [36]:
df_master.filter(pl.col("management_name") == region).head()

pax_counter_id,equipmentname,station_id,tplnr,tpname,station_name,station_ril100,management_name,region_name,federal_state_name,station_longitude,station_latitude,produktlinie,untersegment
str,str,i64,str,str,str,str,str,str,str,f64,f64,str,str
"""d48afc8ec697""","""DSA+_Typ2S_Gl2…",238,"""00238-01-B01""","""Bahnsteig 1, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""a0a3b38ca087""","""DSA+_Typ2S_Gl1…",238,"""00238-01-B01""","""Bahnsteig 1, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""244cab062d53""","""DSA+_Typ2S_Gl4…",238,"""00238-01-B02""","""Bahnsteig 2, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""3494545a211f""","""DSA+_Typ2S_Gl3…",238,"""00238-01-B02""","""Bahnsteig 2, G…","""Babenhausen (H…","""FBA""","""Darmstadt""","""Mitte""","""Hessen""",8.957182,49.957963,"""Knotenbahnhof""","""Regionaler Kno…"
"""a0a3b38c6c4f""","""DSA+_Typ2P_Gl4…",614,"""00614-01-B03""","""Bahnsteig 3, G…","""Biblis""","""FBL""","""Darmstadt""","""Mitte""","""Hessen""",8.450413,49.688881,"""Zubringerbahnh…","""Regionaler Zub…"


In [39]:
cols_of_interest_master = ["pax_counter_id", "station_id", "station_name", "station_longitude", "station_latitude"]

In [48]:
df_jd = df_pax.join(df_master.select(cols_of_interest_master), on = "pax_counter_id", how = "left")
df_jd.head()

pax_counter_id,time_iot,data_pax,station_id,station_name,station_longitude,station_latitude
str,str,i64,i64,str,f64,f64
"""244cab022ddf""","""2024-12-01 00:…",9,5815,"""Seligenstadt (…",8.967472,50.04146
"""244cab022ddf""","""2024-12-01 00:…",5,5815,"""Seligenstadt (…",8.967472,50.04146
"""244cab022ddf""","""2024-12-01 00:…",6,5815,"""Seligenstadt (…",8.967472,50.04146
"""244cab022ddf""","""2024-12-01 00:…",6,5815,"""Seligenstadt (…",8.967472,50.04146
"""244cab022ddf""","""2024-12-01 00:…",8,5815,"""Seligenstadt (…",8.967472,50.04146
